Autor: Esteban Suárez Calvo

In [1]:
from datetime import timedelta
from time import time

from qiskit import QuantumCircuit
from qiskit.circuit.library import (
    RealAmplitudes,
    n_local,
    z_feature_map,
    zz_feature_map,
)
from qiskit_aer import AerSimulator
from qiskit_algorithms.optimizers import Optimizer
from qiskit_ibm_runtime import Sampler
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.optimizers import COBYLA, CRS, SPSA
from qiskit_machine_learning.utils import algorithm_globals
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

# CCAM - Práctica 2: Redes Neuronales Cuánticas

Antes de empezar, fijaremos la semilla aleatoria, de manera que los resultados obtenidos en el notebook sean reproducibles.

In [2]:
algorithm_globals.random_seed = 1

## Preprocesado

### Normalización del dataset

Normalizamos el dataset utilizando la normalización MinMax.

In [3]:
data = load_breast_cancer()
X = data.data
y = data.target
X = MinMaxScaler().fit_transform(X)
print(f"Número de características = {X.shape[1]}")

Número de características = 30


### Reducción de dimensionalidad

Dado que la simulación de circuitos cuánticos escala exponencialmente con el número de qubits, trabajar con 30 características iniciales resulta inviable.

Por lo tango, aplicamos la técnica ***Principal Component Analysis (PCA)*** para  reducir la dimensionalidad a **12 características**.

Aunque esto implica una pérdida de información, es un compromiso necesario para ajustar el problema procesable por el simulador en un tiempo razonable.

Como veremos en los siguientes apartados, dependiendo del optimizador escogido, el tiempo de entrenamiento puede ser bastante elevado (horas), mientras que en otros dura cuestión de minutos. Por lo tanto, aunque resulta posible utilizar un mayor número de características, hay ciertos optimizadores para los cuales no podríamos haber realizado la simulación.

Por lo tanto, al elegir 12 características, estamos buscando un equilibrio entre reducir la dimensionalidad para poder utilizar varios optimizadores diferentes, pero sin eliminar demasiada información.

In [4]:
X = PCA(n_components=12).fit_transform(X)
number_of_features = X.shape[1]
print(f"Nuevo número de características = {number_of_features}")

Nuevo número de características = 12


### Dividir el conjunto entre entrenamiento y test

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.20, random_state=algorithm_globals.random_seed
)
print(f"Datos de entrenamiento: {len(X_train)}")
print(f"Datos de test: {len(X_test)}")

Datos de entrenamiento: 455
Datos de test: 114


### Clasificación con un SVM

Como ya hemos comentado, reducir la dimensionalidad de nuestro dataset conlleva cierta pérdida de información. Si esta pérdida es muy elevada, puede hacer que las clases que queremos clasificar ya no sean distinguibles entre sí y que, por tanto, ningún clasificador (ya sea clásico o cuántico), podrá resolver el problema.

Por este motivo, a continuación realizaremos una clasificación utilizando **máquinas de soporte vectorial (SVM)**, con el fin de comprobar que esto no sucede.

In [23]:
svc = SVC()

start = time()
svc.fit(X_train, Y_train)
total_time = time() - start
print(f"SVC entrenado en {total_time * 1000:.2} ms")

train_score = svc.score(X_train, Y_train)
test_score = svc.score(X_test, Y_test)

print(f"SVC en entrenamiento: {train_score:.5}")
print(f"SVC en test: {test_score:.5}")

SVC entrenado en 6.1 ms
SVC en entrenamiento: 0.98242
SVC en test: 0.97368


Como podemos observar, en cuestión de milisegundos, podemos obtener un `SVC` capaz de resolver el problema prácticamente en la totalidad de los casos.

## Desarrolla una QNN para predecir la clase de cada caso

A continuación, definimos una función que, dado un `feature_map`, un `ansatz` y un `optimizer` arbitrarios, crea el `VQC` correspondiente y lo entrena. Además, también imprime por pantalla el tiempo de entrenamiento.

Definimos también un callback para hacer más interactivo el proceso de entrenamiento, de manera que, a cada iteración, se muestre el número de la iteración y el *loss*:

In [7]:
def train_VQC(
    feature_map: QuantumCircuit, ansatz: QuantumCircuit, optimizer: Optimizer
) -> VQC:
    global X_train, Y_train

    vqc = VQC(
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        callback=LossLogger().log_loss,
        sampler=Sampler(AerSimulator()),
    )

    start = time()
    vqc.fit(X_train, Y_train)
    total_time = time() - start
    print(f"Tiempo total de entrenamiento: {timedelta(seconds=total_time)}")

    return vqc


class LossLogger:
    def __init__(self):
        self._iteration = 0

    def log_loss(self, weights, obj_func_eval):
        self._iteration += 1
        print(f"Iteración {self._iteration} - Loss: {obj_func_eval:.4f}")

Definimos también una función para poder evaluar el clasificador:

In [8]:
def evaluate_VQC(vqc: VQC) -> None:
    global X_train, Y_train

    train_score = vqc.score(X_train, Y_train)
    print(f"SVC en entrenamiento: {train_score:.5}")

    test_score = vqc.score(X_test, Y_test)
    print(f"SVC en test: {test_score:.5}")

## Ejecución con distintos `VQC`

A continuación realizamos una serie de pruebas, con el objetivo de comparar diferentes `VQC`:

- **VQC 1**

In [9]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 1.0023
Iteración 2 - Loss: 0.9983
Iteración 3 - Loss: 1.0004
Iteración 4 - Loss: 0.9853
Iteración 5 - Loss: 0.9817
Iteración 6 - Loss: 0.9839
Iteración 7 - Loss: 0.9804
Iteración 8 - Loss: 0.9800
Iteración 9 - Loss: 0.9804
Iteración 10 - Loss: 0.9834
Iteración 11 - Loss: 0.9873
Iteración 12 - Loss: 0.9855
Iteración 13 - Loss: 0.9830
Iteración 14 - Loss: 0.9813
Iteración 15 - Loss: 1.0110
Iteración 16 - Loss: 0.9834
Iteración 17 - Loss: 0.9812
Iteración 18 - Loss: 0.9849
Iteración 19 - Loss: 0.9806
Iteración 20 - Loss: 0.9799
Tiempo total de entrenamiento: 0:04:30.873468
SVC en entrenamiento: 0.55604
SVC en test: 0.57895


- **VQC 2**

In [24]:
feature_map = z_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9851
Iteración 2 - Loss: 1.0049
Iteración 3 - Loss: 0.9828
Iteración 4 - Loss: 0.9732
Iteración 5 - Loss: 0.9739
Iteración 6 - Loss: 0.9751
Iteración 7 - Loss: 0.9740
Iteración 8 - Loss: 0.9790
Iteración 9 - Loss: 0.9745
Iteración 10 - Loss: 0.9770
Iteración 11 - Loss: 0.9765
Iteración 12 - Loss: 0.9719
Iteración 13 - Loss: 0.9762
Iteración 14 - Loss: 0.9809
Iteración 15 - Loss: 0.9960
Iteración 16 - Loss: 0.9765
Iteración 17 - Loss: 0.9710
Iteración 18 - Loss: 0.9722
Iteración 19 - Loss: 0.9709
Iteración 20 - Loss: 0.9732
Tiempo total de entrenamiento: 0:04:31.010642
SVC en entrenamiento: 0.6022
SVC en test: 0.53509


- **VQC 3**

In [11]:
feature_map = zz_feature_map(number_of_features)
ansatz = RealAmplitudes(number_of_features).decompose()
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9901
Iteración 2 - Loss: 0.9930
Iteración 3 - Loss: 0.9971
Iteración 4 - Loss: 0.9951
Iteración 5 - Loss: 0.9980
Iteración 6 - Loss: 0.9970
Iteración 7 - Loss: 0.9888
Iteración 8 - Loss: 0.9991
Iteración 9 - Loss: 0.9925
Iteración 10 - Loss: 0.9942
Iteración 11 - Loss: 0.9975
Iteración 12 - Loss: 0.9942
Iteración 13 - Loss: 0.9944
Iteración 14 - Loss: 1.0002
Iteración 15 - Loss: 0.9945
Iteración 16 - Loss: 0.9918
Iteración 17 - Loss: 0.9959
Iteración 18 - Loss: 0.9960
Iteración 19 - Loss: 0.9953
Iteración 20 - Loss: 0.9936
Tiempo total de entrenamiento: 0:04:30.139605
SVC en entrenamiento: 0.48571
SVC en test: 0.51754


- **VQC 4**

In [25]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear", reps=3)
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 1.0032
Iteración 2 - Loss: 0.9912
Iteración 3 - Loss: 0.9920
Iteración 4 - Loss: 0.9865
Iteración 5 - Loss: 0.9839
Iteración 6 - Loss: 0.9857
Iteración 7 - Loss: 0.9843
Iteración 8 - Loss: 0.9822
Iteración 9 - Loss: 0.9870
Iteración 10 - Loss: 0.9853
Iteración 11 - Loss: 0.9848
Iteración 12 - Loss: 0.9873
Iteración 13 - Loss: 0.9864
Iteración 14 - Loss: 0.9699
Iteración 15 - Loss: 0.9785
Iteración 16 - Loss: 0.9675
Iteración 17 - Loss: 0.9677
Iteración 18 - Loss: 0.9687
Iteración 19 - Loss: 0.9680
Iteración 20 - Loss: 0.9676
Tiempo total de entrenamiento: 0:04:32.633151
SVC en entrenamiento: 0.61319
SVC en test: 0.57895


- **VQC 5**

In [26]:
feature_map = z_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear", reps=3)
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9822
Iteración 2 - Loss: 0.9988
Iteración 3 - Loss: 0.9925
Iteración 4 - Loss: 0.9930
Iteración 5 - Loss: 0.9877
Iteración 6 - Loss: 0.9827
Iteración 7 - Loss: 0.9848
Iteración 8 - Loss: 0.9820
Iteración 9 - Loss: 0.9778
Iteración 10 - Loss: 0.9841
Iteración 11 - Loss: 0.9783
Iteración 12 - Loss: 0.9850
Iteración 13 - Loss: 0.9816
Iteración 14 - Loss: 0.9894
Iteración 15 - Loss: 0.9974
Iteración 16 - Loss: 0.9803
Iteración 17 - Loss: 0.9844
Iteración 18 - Loss: 0.9793
Iteración 19 - Loss: 0.9839
Iteración 20 - Loss: 0.9843
Tiempo total de entrenamiento: 0:04:33.557581
SVC en entrenamiento: 0.59121
SVC en test: 0.54386


- **VQC 6**

In [9]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = CRS()

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Tiempo total de entrenamiento: 3:46:17.754241
SVC en entrenamiento: 0.6021978021978022
SVC en test: 0.6228070175438597


- **VQC 7**

In [14]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = SPSA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Tiempo total de entrenamiento: 0:20:46.916321
SVC en entrenamiento: 0.6153846153846154
SVC en test: 0.6491228070175439


## Resultados

Tenemos entonces los siguientes resultados:


| Clasificador | `feature_map`    | `ansatz`          | `optimizer` | Accuracy en entrenamiento | Accuracy en test | Tiempo de entrenamiento |
| ------------ | ---------------- | ----------------- | ----------- | ------------------------- | ---------------- | ----------------------- |
| **VQC 1**    | `zz_feature_map` | `n_local`         | `COBYLA`    | 0.55604                   | 0.57895          | 0:04:30                 |
| **VQC 2**    | `z_feature_map`  | `n_local`         | `COBYLA`    | 0.6022                    | 0.53509          | 0:04:31                 |
| **VQC 3**    | `zz_feature_map` | `RealAmplitudes`  | `COBYLA`    | 0.48571                   | 0.51754          | 0:04:30                 |
| **VQC 4**    | `zz_feature_map` | `n_local(reps=3)` | `COBYLA`    | 0.61319                   | 0.57895          | 0:04:32                 |
| **VQC 5**    | `z_feature_map`  | `n_local(reps=3)` | `COBYLA`    | 0.59121                   | 0.54386          | 0:04:33                 |
| **VQC 6**    | `zz_feature_map` | `n_local`         | `CRS`       | 0.60219                   | 0.62280          | 3:46:17                 |
| **VQC 7**    | `zz_feature_map` | `n_local`         | `SPSA`      | 0.61538                   | 0.64912          | 0:20:46                 |


## Conclusiones

Los resultados muestran una diferencia muy notoria, mientras que el SVC clásico alcanza un accuracy cercano al 100% en test en milisegundos, el mejor de los VQC ni siquiera llega al 65% en test, con una función de loss constante que no es capaz de converger.

Este comportamiento no indica necesariamente que el VQC sea incapaz de resolver el problema, sino que todo apunta a un problema de entrenabilidad conocido como ***Barren Plateaus***.

Este problema consiste en que el gradiente de la función de coste tiende a desvanecerse. Es decir, el optimizador navega por una superficie plana donde no encuentra dirección de descenso, lo que impide el aprendizaje.

Finalmente, el éxito del SVC con solo 12 características nos confirma que los datos son separables en el espacio clásico reducido. La gran fortaleza de los VQC es proyectar datos complejos a un espacio de Hilbert de alta dimensionalidad para facilitar su separación, pero en este caso esa proyección resulta redundante. Por lo tanto, cuando los datos no tienen una complejidad estructural alta, usar VQC supone una sobrecarga computacional inmensa que no aporta una ventaja real frente a métodos clásicos.